In [1]:
%load_ext autoreload
%autoreload 2
%aimport anndata
%aimport os
%aimport torch
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from typing import Dict, Union, List, Tuple
from torch.utils.data import Dataset
# Control UMAP numba warnings
import warnings; warnings.simplefilter('ignore')
import copy
import scvi
from scvi.dataset import CortexDataset, RetinaDataset, CsvDataset
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi import set_seed
from scvi.dataset.utils import setup_anndata

#STUFF THAT I MESSED AROUND WITH
# from BioDataset import BioDataset
# from constants import X_KEY, BATCH_KEY, LOCAL_L_MEAN_KEY, LOCAL_L_VAR_KEY, LABELS_KEY

# Sets torch and numpy random seeds, run after all scvi imports
set_seed(0)


In [2]:
from scvi.dataset import  DownloadableAnnDataset, GeneExpressionDataset
save_path = "data/"
dataset1 = DownloadableAnnDataset(
    "pbmc_10k_protein.h5ad",
    save_path=save_path,
    url="https://github.com/YosefLab/scVI-data/raw/master/pbmc_10k_protein_v3.h5ad?raw=true",
    cell_measurements_col_mappings={"protein_expression":"protein_names"},
)

dataset2 = DownloadableAnnDataset(
    "pbmc_5k_protein.h5ad",
    save_path=save_path,
    url="https://github.com/YosefLab/scVI-data/raw/master/pbmc_5k_protein_v3.h5ad?raw=true",
    cell_measurements_col_mappings={"protein_expression":"protein_names"},
)

[2020-06-23 11:47:30,841] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/pbmc_10k_protein.h5ad already downloaded
[2020-06-23 11:47:33,137] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-23 11:47:33,139] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-23 11:47:34,607] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-23 11:47:34,993] INFO - scvi.dataset.dataset | Downsampled from 6855 to 6855 cells
[2020-06-23 11:47:35,045] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/pbmc_5k_protein.h5ad already downloaded
[2020-06-23 11:47:36,263] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-23 11:47:36,265] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-23 11:47:37,130] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-23 11:47:37,353] INFO - scvi.dataset.dataset | Downsampled from 3994 to 3994 cells


In [3]:
from scvi.models import TOTALVI
from scvi.inference import TotalPosterior, TotalTrainer

In [4]:
# concatenate the datasets
dataset = GeneExpressionDataset()
dataset.populate_from_datasets([dataset1, dataset2])
dataset.subsample_genes(4000)

[2020-06-23 11:47:37,440] INFO - scvi.dataset.dataset | Merging datasets. Input objects are modified in place.
[2020-06-23 11:47:37,440] INFO - scvi.dataset.dataset | Gene names and cell measurement names are assumed to have a non-null intersection between datasets.
[2020-06-23 11:47:37,464] INFO - scvi.dataset.dataset | Keeping 15792 genes
[2020-06-23 11:47:39,990] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-23 11:47:41,363] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-23 11:47:41,364] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-23 11:47:42,739] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-23 11:47:43,374] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-23 11:47:43,375] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-23 11:47:47,117] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-23 11:47:47,119] INFO - 

In [5]:
adata = dataset.to_anndata()

In [6]:
setup_anndata(adata, protein_expression_obsm_key = 'protein_expression', batch_key = 'batch_indices')

[2020-06-23 11:47:57,668] INFO - scvi.dataset.utils | Using data from adata.X
[2020-06-23 11:47:57,669] INFO - scvi.dataset.utils | Using batches from adata.obs["batch_indices"]
[2020-06-23 11:47:57,670] INFO - scvi.dataset.utils | No label_key inputted, assuming all cells have same label
[2020-06-23 11:47:57,674] INFO - scvi.dataset.utils | Computing library size prior per batch
[2020-06-23 11:47:57,763] INFO - scvi.dataset.utils | Successfully registered anndata object containing 10849 cells, 4000 genes, and 2 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels', 'protein_expression']


In [7]:
test_mode = False

totalvae = TOTALVI(
    adata.uns['scvi_summary_stats']['n_genes'], 
    adata.uns['scvi_summary_stats']['n_proteins'], 
    0
#     adata.uns['scvi_summary_stats']['n_batch'],
)
use_cuda = True
lr = 4e-3
n_epochs = 500

# totalVI is trained on 90% of the data
# Early stopping does not comply with our automatic notebook testing so we disable it when testing
trainer = TotalTrainer(
    totalvae,
    adata,
    train_size=0.90,
    test_size=0.10,
    use_cuda=use_cuda,
    frequency=1,
    batch_size=256,
    early_stopping_kwargs="auto" if not test_mode else None
)

In [ ]:
trainer.train(n_epochs = 10)